<a href="https://colab.research.google.com/github/stevegbrooks/big-portfolio-learner/blob/big-data/step1b_clean_and_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 545 Final Project

## Big Portfolio Learner: Clean Up Technical Indicators Data and Combine with Price Data

### Team members: Steven Brooks & Chenlia Xu

In [1]:
import random
import numpy as np 
import json
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os

In [2]:
%%capture
## If boto3 not already installed uncomment the following:
!pip3 install boto3

In [3]:
import boto3
from botocore import UNSIGNED

from botocore.config import Config

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
s3.Bucket('cis545project').download_file('data/stock_data_2002_2019.csv', 'stock_data_2002_2019.csv')
s3.Bucket('cis545project').download_file('data/technical_data.zip', 'technical_data.zip')

In [4]:
%%capture

tech_dir = "technical_data"
if not os.path.exists(tech_dir):
  os.makedirs(tech_dir)
!unzip /content/technical_data.zip -d /content/$tech_dir
!rm -f $tech_dir/.gitempty

# Setup Spark Session

In [5]:
%%capture

!apt install libkrb5-dev
!pip install findspark
!pip install sparkmagic
!pip install pyspark
!pip install pyspark --user
!pip install seaborn --user
!pip install imageio --user
!pip install folium --user

!apt update
!apt install gcc python-dev libkrb5-dev

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.getOrCreate()

In [7]:
%load_ext sparkmagic.magics

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [8]:
#graph section
import networkx as nx
# SQLite RDBMS
import sqlite3
# Parallel processing
# import swifter
import pandas as pd
# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = '/content/spark-2.4.5-bin-hadoop2.7'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import pyspark
from pyspark.sql import SQLContext

try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)

We will load the data into the Spark context here.

In [11]:
stock_data_sdf = spark.read.load(
    'stock_data_2002_2019.csv', 
    format = 'csv', 
    header = 'true', 
    inferSchema = 'true', 
    sep = ','
)

stock_data_sdf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: long (nullable = true)
 |-- _c4: timestamp (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: string (nullable = true)



In [10]:
technical_data_sdf = spark.read.load(
    'technical_data/*.csv', 
    format = 'csv', 
    header = 'true', 
    inferSchema = 'true', 
    sep = ','
)

technical_data_sdf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- EMA: string (nullable = true)
 |-- MACD: string (nullable = true)
 |-- MACD_Hist: string (nullable = true)
 |-- MACD_Signal: string (nullable = true)
 |-- SlowD: string (nullable = true)
 |-- SlowK: string (nullable = true)
 |-- RSI: string (nullable = true)
 |-- Real_Lower_Band: string (nullable = true)
 |-- Real_Middle_Band: string (nullable = true)
 |-- Real_Upper_Band: double (nullable = true)

